In [26]:
SOURCE = 'SpamData/01_Processing/practice_email.txt'

SPAM_PATH_1 = 'SpamData/01_Processing/spam_assassin_corpus/spam_1'
SPAM_PATH_2 = 'SpamData/01_Processing/spam_assassin_corpus/spam_2'
HAM_PATH_1 = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_1'
HAM_PATH_2 = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_2'

SPAM_CATEGORY = 1
HAM_CATEGORY = 0


In [16]:
from os import walk
from os.path import join
import pandas as pd

In [19]:
data = open(SOURCE, encoding='latin-1')

is_email_body = False
lines = []

for line in data:
    if is_email_body:
        lines.append(line)
    elif line == '\n':
        is_email_body = True
        
data.close()

email_body = '\n'.join(lines)
print(email_body)



Dear Mr Still



Good tidings to you and all your staff for the festive season ahead (Christmas).

Now to the crux of the matter-in-hand: I am a fully qualified Santa Claus and am wondering whether you might consider me to run my own "Santa's Grotto" in your store.

But WAIT! You're probably thinking: "What makes him so special?"

Well, first of all, I have made several changes to the characterisation of Father Christmas. Rather than greeting the children with shouts of "Ho, ho, ho!" I prefer to whisper the phrase "Dependence is not unfathomable in this cruel world we live in". In addition, my gifts are ALL hand-made, ranging from felt hoops to vanilla-pod holders.

You will note also, from the enclosed sketch, that I have radically redesigned Santa's outfit and have renamed my character "Lord Buckles". Would you be interested in employing me? I promise NEVER to let you down.

I look forward to hearing from you.



Best wishes

Robin Cooper

[Excerpt from the book: The Timewaster Let

In [22]:
def combine_email_body_generator(path):
    
    for root, dirnames, filenames in walk(path):
        for file_name in filenames:
            
            file = join(root, file_name)
            
            data = open(file, encoding='latin-1')

            is_email_body = False
            lines = []

            for line in data:
                if is_email_body:
                    lines.append(line)
                elif line == '\n':
                    is_email_body = True

            data.close()

            email_body = '\n'.join(lines)
            
            yield file_name, email_body

In [23]:
def directory_list(path, classification):
    rows = []
    row_names = []
    
    for file_name, email_body in combine_email_body_generator(path):
        rows.append({'MESSAGE': email_body, 'CLASS': classification})
        row_names.append(file_name)
        
    return pd.DataFrame(rows, index=row_names)

In [28]:
spam_emails = directory_list(SPAM_PATH_1, SPAM_CATEGORY)
spam_emails = spam_emails.append(directory_list(SPAM_PATH_2, SPAM_CATEGORY))
spam_emails.shape

(1898, 2)

In [29]:
ham_emails = directory_list(HAM_PATH_1, HAM_CATEGORY)
ham_emails = ham_emails.append(directory_list(HAM_PATH_2, HAM_CATEGORY))
ham_emails.shape

(3901, 2)

In [30]:
df = pd.concat([spam_emails, ham_emails])
df.shape
df.head()

,MESSAGE,CLASS
00249.5f45607c1bffe89f60ba1ec9f878039a,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",1
00373.ebe8670ac56b04125c25100a36ab0510,ATTENTION: This is a MUST for ALL Computer Use...,1
00214.1367039e50dc6b7adb0f2aa8aba83216,This is a multi-part message in MIME format.\n...,1
00210.050ffd105bd4e006771ee63cabc59978,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,1
00033.9babb58d9298daa2963d4f514193d7d6,This is the bottom line. If you can GIVE AWAY...,1


In [33]:
# check if any message value is null
df.MESSAGE.isnull().values.any()

False

In [34]:
# check if there are any empty email (length = 0)
(df.MESSAGE.str.len() == 0).any()

True

In [35]:
(df.MESSAGE.str.len() == 0).sum()

3

In [37]:
df[df.MESSAGE.str.len() == 0].index

Index(['cmds', 'cmds', 'cmds'], dtype='object')

## Remove system file entries from the dataframe

In [38]:
 df.drop(['cmds'], inplace=True)

In [39]:
df.shape

(5796, 2)

In [40]:
document_ids = range(0, len(df.index))
df['DOC_ID'] = document_ids

In [41]:
df['FILE_NAME'] = df.index
df.set_index('DOC_ID', inplace=True)
df.head()

,MESSAGE,CLASS,FILE_NAME
DOC_ID,,,
0,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",1,00249.5f45607c1bffe89f60ba1ec9f878039a
1,ATTENTION: This is a MUST for ALL Computer Use...,1,00373.ebe8670ac56b04125c25100a36ab0510
2,This is a multi-part message in MIME format.\n...,1,00214.1367039e50dc6b7adb0f2aa8aba83216
3,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,1,00210.050ffd105bd4e006771ee63cabc59978
4,This is the bottom line. If you can GIVE AWAY...,1,00033.9babb58d9298daa2963d4f514193d7d6
